---
Setup

In [1]:
!cat /home/project/ml/pytorch/torch/version.py

__version__ = '1.10.0a0+git5c15050'
debug = False
cuda = None
git_version = '5c1505076bfa764088e2ccef19d7f18336084530'
hip = None


In [2]:
!PYTHONPATH=/home/project/ml/pytorch/ python -c "import torch; print(torch.__file__, torch._C._GLIBCXX_USE_CXX11_ABI)"

/home/project/ml/pytorch/torch/__init__.py True


In [3]:
#pragma cling add_include_path("/home/project/ml/pytorch/torch/include")
#pragma cling add_include_path("/home/project/ml/pytorch/torch/include/torch/csrc/api/include")
// If you want to add library path
#pragma cling add_library_path("/home/project/ml/pytorch/torch/lib")
// If you want to load library
#pragma cling load("libtorch")
#pragma cling load("libtorch_cpu")
#pragma cling load("libc10")

----

Test

In [4]:
#include <iostream>
#include <ATen/ATen.h>

auto p = at::CPU(at::kFloat);
std::cout << p << "\n";
auto t = at::ones({3, 10}, p);
std::cout << t << "\n";

CPUFloatType
 1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1
[ CPUFloatType{3,10} ]


In [5]:
t.sizes()[1]

10

In [6]:
t.sizes().size()

2

----

Iterate over input (B, C, H, w), output (B, C, h, w), weights (1, 1, h * M, 1)

In [7]:
#include <vector>
#include <ATen/native/TensorIterator.h>

In [13]:
at::Tensor input = at::arange(2 * 3 * 10 * 5, at::CPU(at::kFloat)).reshape({2, 3, 10, 5});
at::Tensor output = at::zeros({2, 3, 4, 5});

int ndims = input.dim();
int reshape_dim = 2;
int output_size = output.sizes()[reshape_dim];

// Restride input
{
    auto shape = input.sizes().vec();
    auto strides = input.strides().vec();
    auto oshape = output.sizes();

    for (int i=2; i<ndims; i++) {
        shape[i] = oshape[i];
        strides[i] = 0;
    }
    input = input.as_strided(shape, strides);
}

// Define indices
at::Tensor indices;
auto new_shape = std::vector<int64_t>(ndims);
for (int j=0; j<new_shape.size(); j++) {
    new_shape[j] = 1;
}
new_shape[reshape_dim] = output_size;

indices = at::arange(new_shape[reshape_dim], at::CPU(at::kLong)).reshape(new_shape);
indices *= (int64_t) sizeof(float);

In [14]:
std::cout << "-- Input strides: " << input.strides() << std::endl;

-- Input strides: [150, 50, 0, 0]


In [15]:
std::cout << "-- Indices strides: " << indices.strides() << std::endl;

-- Indices strides: [4, 4, 1, 1]


In [20]:
std::cout << input.sizes()

[2, 3, 4, 5]

@0x7f3de4251b60

In [24]:
auto iter = at::TensorIteratorConfig()
    .add_output(output)
    .add_input(input)
    .add_input(indices)    
    .check_all_same_dtype(false)    
    .declare_static_dtype_and_device(input.scalar_type(), input.device())
    .resize_outputs(false)
//     .declare_static_shape(input.sizes(), /*squash_dims=*/{2, 3})
    .build();

In [26]:
iter.numel()

120

In [25]:
auto test_loop = [&](char **data, const int64_t* strides, int64_t n) {

    std::cout << "n : " << n << std::endl;
    std::cout << "Output stride: " << strides[0] << std::endl;
    std::cout << "Input stride: " << strides[1] << std::endl;
    std::cout << "Indices stride: " << strides[2] << std::endl;

    auto * out = data[0];
    auto * in = data[1];
    auto * idx = data[2];    
    
    // assume float data type for this example.
    std::cout << " - input data: " << std::endl;
    for (int i = 0; i < n; i++) {
        std::cout << *reinterpret_cast<float*>(&in[i * strides[1] + idx[i * strides[2]]]) << " ";
    }
    std::cout << std::endl;
    
    std::cout << " - indices data: " << std::endl;
    for (int i = 0; i < n; i++) {
        std::cout << *reinterpret_cast<long*>(&idx[i * strides[2]]) << " ";
    }
    std::cout << std::endl;
    
};


iter.for_each(test_loop);

n : 5
Output stride: 4
Input stride: 0
Indices stride: 0
 - input data: 
0 0 0 0 0 
 - indices data: 
0 0 0 0 0 
n : 5
Output stride: 4
Input stride: 0
Indices stride: 0
 - input data: 
1 1 1 1 1 
 - indices data: 
4 4 4 4 4 
n : 5
Output stride: 4
Input stride: 0
Indices stride: 0
 - input data: 
2 2 2 2 2 
 - indices data: 
8 8 8 8 8 
n : 5
Output stride: 4
Input stride: 0
Indices stride: 0
 - input data: 
3 3 3 3 3 
 - indices data: 
12 12 12 12 12 
n : 5
Output stride: 4
Input stride: 0
Indices stride: 0
 - input data: 
50 50 50 50 50 
 - indices data: 
0 0 0 0 0 
n : 5
Output stride: 4
Input stride: 0
Indices stride: 0
 - input data: 
51 51 51 51 51 
 - indices data: 
4 4 4 4 4 
n : 5
Output stride: 4
Input stride: 0
Indices stride: 0
 - input data: 
52 52 52 52 52 
 - indices data: 
8 8 8 8 8 
n : 5
Output stride: 4
Input stride: 0
Indices stride: 0
 - input data: 
53 53 53 53 53 
 - indices data: 
12 12 12 12 12 
n : 5
Output stride: 4
Input stride: 0
Indices stride: 0
 - input 

---
Compare 2 implementations

In [6]:
namespace at {
namespace native {


// Helper structs to use with ti_upsample_generic_Nd_kernel_impl
template <typename index_t, typename scalar_t>
struct HelperInterpBase {

  template <typename filter_fn_t>
  static inline std::vector<Tensor> _compute_indices_weights_aa(
      int64_t input_size,
      int64_t output_size,
      int64_t stride,
      int64_t ndims,
      int64_t reshape_dim,
      bool align_corners,
      scalar_t scale,
      int& in_out_interp_size,
      filter_fn_t filter_fn
    ) {

    int interp_size = in_out_interp_size;
    scalar_t support =
        (scale >= 1.0) ? (interp_size * 0.5) * scale : interp_size * 0.5;
    interp_size = (int)ceilf(support) * 2 + 1;

    // return interp_size
    in_out_interp_size = interp_size;

    std::vector<Tensor> output;
    auto new_shape = std::vector<int64_t>(ndims, 1);
    new_shape[reshape_dim] = output_size;

    // ---- Bounds approach as in PIL -----
    // bounds: xmin/xmax
    output.emplace_back(
        empty(new_shape, CPU(c10::CppTypeToScalarType<index_t>())));
    output.emplace_back(
        empty(new_shape, CPU(c10::CppTypeToScalarType<index_t>())));
    output.emplace_back(
        empty(new_shape, CPU(c10::CppTypeToScalarType<index_t>())));

    std::cout << "support: " << support << std::endl;
    std::cout << "interp_size: " << interp_size << std::endl;

    {
      // Weights
      new_shape[reshape_dim] = output_size * interp_size;
      std::cout << "new_shape: " << output_size * interp_size << std::endl;
      auto wts = empty(new_shape, CPU(c10::CppTypeToScalarType<scalar_t>()));
      auto strides = wts.strides().vec();
      strides[reshape_dim] = 0;
      new_shape[reshape_dim] = output_size;
      wts = wts.as_strided(new_shape, strides);
      output.emplace_back(wts);
      // Weights indices
      output.emplace_back(
          empty(new_shape, CPU(c10::CppTypeToScalarType<index_t>())));
    }

    scalar_t center, total_w, invscale = (scale >= 1.0) ? 1.0 / scale : 1.0;
    index_t zero = static_cast<index_t>(0);
    int64_t* idx_ptr_xmin = output[0].data_ptr<index_t>();
    int64_t* idx_ptr_size = output[1].data_ptr<index_t>();
    int64_t* idx_ptr_stride = output[2].data_ptr<index_t>();
    scalar_t* wt_ptr = output[3].data_ptr<scalar_t>();
    int64_t* wt_idx_ptr = output[4].data_ptr<index_t>();

    int64_t xmin, xmax, j;

    for (int64_t i = 0; i < output_size; i++) {
      center = scale * (i + 0.5);
      xmin = std::max(static_cast<int64_t>(center - support + 0.5), zero);
      xmax =
          std::min(static_cast<int64_t>(center + support + 0.5), input_size) -
          xmin;
      idx_ptr_xmin[i] = xmin * stride;
      idx_ptr_size[i] = xmax;
      idx_ptr_stride[i] = stride;

      wt_idx_ptr[i] = i * interp_size * sizeof(scalar_t);

      total_w = 0.0;
      for (j = 0; j < xmax; j++) {
        scalar_t w = filter_fn((j + xmin - center + 0.5) * invscale);
        wt_ptr[i * interp_size + j] = w;
        total_w += w;
      }
      for (j = 0; j < xmax; j++) {
        if (total_w != 0.0) {
          wt_ptr[i * interp_size + j] /= total_w;
        }
      }

      for (; j < interp_size; j++) {
        wt_ptr[i * interp_size + j] = static_cast<scalar_t>(0.0);
      }
    }
    return output;
  }

};

}
}

In [7]:
namespace at {
namespace native {

    
template <typename scalar_t>
static inline scalar_t compute_scales_value(
    const c10::optional<double> scale,
    int64_t input_size,
    int64_t output_size) {
      // see Note [compute_scales_value]
      // FIXME: remove magic > 0 after we ensure no models were serialized with -1 defaults.
      return (scale.has_value() && scale.value() > 0.)
          ? static_cast<scalar_t>(1.0 / scale.value())
          : (static_cast<scalar_t>(input_size) / output_size);
}    
    

template <typename scalar_t>
static inline scalar_t area_pixel_compute_scale(
    int64_t input_size,
    int64_t output_size,
    bool align_corners,
    const c10::optional<double> scale) {
  // see Note [area_pixel_compute_scale]
  if (output_size > 1) {
    return align_corners
        ? static_cast<scalar_t>(input_size - 1) / (output_size - 1)
        : compute_scales_value<scalar_t>(scale, input_size, output_size);
  } else {
    return scalar_t(0);
  }
}   

}
}

In [8]:
namespace at {
namespace native {

    
template <typename index_t, typename scalar_t>
struct HelperInterpLinear : public HelperInterpBase<index_t, scalar_t> {
  static const int interp_size = 2;

  // taken from
  // https://github.com/python-pillow/Pillow/blob/6812205f18ca4ef54372e87e1a13ce4a859434df/
  // src/libImaging/Resample.c#L20-L29
  static inline scalar_t _filter(scalar_t x) {
    if (x < 0.0) {
      x = -x;
    }
    if (x < 1.0) {
      return 1.0 - x;
    }
    return 0.0;
  }

  static inline std::vector<Tensor> compute_indices_weights(
      int64_t input_size,
      int64_t output_size,
      int64_t stride,
      int64_t ndims,
      int64_t reshape_dim,
      bool align_corners,
      const c10::optional<double> opt_scale,
      bool antialias,
      int& out_interp_size) {

    TORCH_INTERNAL_ASSERT(antialias);
    scalar_t scale = area_pixel_compute_scale<scalar_t>(
        input_size, output_size, align_corners, opt_scale);

    std::cout << "scale: " << scale << std::endl;

    out_interp_size = HelperInterpLinear<index_t, scalar_t>::interp_size;
    return HelperInterpLinear<index_t, scalar_t>::_compute_indices_weights_aa(
        input_size,
        output_size,
        stride,
        ndims,
        reshape_dim,
        align_corners,
        scale,
        out_interp_size,
        _filter);
  }
};

    
}
}

In [9]:
int out_interp_size;
auto expected_result = at::native::HelperInterpLinear<int64_t, float>::compute_indices_weights(64, 10, 5, 4, 2, false, c10::nullopt, true, out_interp_size);

scale: 6.4
support: 6.4
interp_size: 15
new_shape: 150


In [10]:
expected_result.size()

5

In [27]:
std::cout << expected_result[0];

(1,1,.,.) = 
    0
   15
   50
   80
  110
  145
  175
  210
  240
  270
[ CPULongType{1,1,10,1} ]

In [46]:
float * w_ptr = (float *) expected_result[3].data_ptr();

int o = 0; 

for (int i=0; i<out_interp_size; i++) {
    std::cout << w_ptr[o * out_interp_size + i] << " ";
}


0.103352 0.131285 0.159218 0.170391 0.142458 0.114525 0.0865922 0.0586592 0.0307263 0.0027933 0 0 0 0 0 

In [12]:
#include <ATen/AccumulateType.h>

In [13]:
auto t_weights = at::empty({out_interp_size});
auto weights = t_weights.accessor<float, 1>();
float support = 6.4;
float scale = 6.4;

In [40]:
template <typename scalar_t>
static inline scalar_t bilinear_filter(scalar_t x) {
    if (x < 0.0) {
      x = -x;
    }
    if (x < 1.0) {
      return 1.0 - x;
    }
    return 0.0;
}


template <typename scalar_t, typename accscalar_t>
static void compute_weights(
    const int64_t i,
    const int64_t input_size,
    const accscalar_t scale,
    const accscalar_t support,
    at::TensorAccessor<scalar_t, 1> weights,
    int64_t & xmin,
    int64_t & xmax) {

  int64_t interp_size = weights.size(0);
  accscalar_t invscale = (scale >= 1.0) ? 1.0 / scale : 1.0;
  accscalar_t center = scale * (i + 0.5);
  xmin = fmax(static_cast<int64_t>(center - support + 0.5), static_cast<int64_t>(0));
  xmax = fmin(static_cast<int64_t>(center + support + 0.5), input_size) - xmin;

  accscalar_t total_w = 0.0;
  int64_t j = 0;
  for (j = 0; j < xmax; j++) {
    accscalar_t w = bilinear_filter((j + xmin - center + 0.5) * invscale);
    weights[j] = static_cast<scalar_t>(w);
    total_w += w;
  }
  for (j = 0; j < xmax; j++) {
    if (total_w != 0.0) {
      weights[j] /= total_w;
    }
  }
  for (; j < interp_size; j++) {
    weights[j] = static_cast<scalar_t>(0.0);
  }
}

In [41]:
int64_t xmin, xsize;
compute_weights(2, 64, scale, support, weights, xmin, xsize);

In [42]:
xsize

12

In [43]:
float * w_ptr = (float *) t_weights.data_ptr();

for (int i=0; i<out_interp_size; i++) {
    std::cout << w_ptr[i] << " ";
}


0.0220588 0.0465686 0.0710784 0.0955882 0.120098 0.144608 0.144608 0.120098 0.0955882 0.0710784 0.0465686 0.0220588 0 0 0 

In [45]:
std::cout << t_weights[0];

0.0220588
[ CPUFloatType{} ]

In [47]:
float support = 100.0;
int interp_size = (int)ceilf(support) * 2 + 1;
interp_size

201

In [69]:
template<typename scalar_t>
scalar_t pow2(scalar_t x) {
    return x * x;
}

In [70]:
template<typename scalar_t>
scalar_t pow3(scalar_t x) {
    return x * x * x;
}

In [71]:
template<typename scalar_t, typename func_t>
scalar_t apply(scalar_t x, func_t func) {
    return func(x) + 1.2;
}

In [72]:
template<typename scalar_t, int ptype>
scalar_t compute(scalar_t x) {

    typedef scalar_t(*func_t)(scalar_t);
    if (ptype == 2) {        
        return apply<scalar_t, func_t>(x, pow2);
    } else if (ptype == 3) {
        return apply<scalar_t, func_t>(x, pow3);
    }
    return -1.0;
}

In [73]:
compute<float, 2>(2.0f)

5.20000f

In [75]:
compute<int, 2>(3)

10